<a href="https://colab.research.google.com/github/diliprc96/Dynamic-Programming/blob/main/FrozenLake_using_Dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group No 160

## Group Member Names:
1. Dilip R 2023ad05030
2.
3.


1.**Problem statement**:

* Develop a reinforcement learning agent using dynamic programming to solve the Treasure Hunt problem in a FrozenLake environment. The agent must learn the optimal policy for navigating the lake while avoiding holes and maximizing its treasure collection.

2.**Scenario**:
* A treasure hunter is navigating a slippery 5x5 FrozenLake grid. The objective is to navigate through the lake collecting treasures while avoiding holes and ultimately reaching the exit (goal).
Grid positions on a 5x5 map with tiles labeled as S, F, H, G, T. The state includes the current position of the agent and whether treasures have been collected.


#### Objective
* The agent must learn the optimal policy π* using dynamic programming to maximize its cumulative reward while navigating the lake.

#### About the environment

The environment consists of several types of tiles:
* Start (S): The initial position of the agent, safe to step.
* Frozen Tiles (F): Frozen surface, safe to step.
* Hole (H): Falling into a hole ends the game immediately (die, end).
* Goal (G): Exit point; reaching here ends the game successfully (safe, end).
* Treasure Tiles (T): Added to the environment. Stepping on these tiles awards +5 reward but does not end the game.

After stepping on a treasure tile, it becomes a frozen tile (F).
The agent earns rewards as follows:
* Reaching the goal (G): +10 reward.
* Falling into a hole (H): -10 reward.
* Collecting a treasure (T): +5 reward.
* Stepping on a frozen tile (F): 0 reward.

#### States
* Current position of the agent (row, column).
* A boolean flag (or equivalent) for whether each treasure has been collected.

#### Actions
* Four possible moves: up, down, left, right

#### Rewards
* Goal (G): +10.
* Treasure (T): +5 per treasure.
* Hole (H): -10.
* Frozen tiles (F): 0.

#### Environment
Modify the FrozenLake environment in OpenAI Gym to include treasures (T) at certain positions. Inherit the original FrozenLakeEnv and modify the reset and step methods accordingly.
Example grid:




**Expected Outcomes:**
1.	Create the custom environment by modifying the existing “FrozenLakeNotSlippery-v0” in OpenAI Gym and Implement the dynamic programming using value iteration and policy improvement to learn the optimal policy for the Treasure Hunt problem.
2.	Calculate the state-value function (V*) for each state on the map after learning the optimal policy.
3.	Compare the agent’s performance with and without treasures, discussing the trade-offs in reward maximization.
4.	Visualize the agent’s direction on the map using the learned policy.
5.	Calculate expected total reward over multiple episodes to evaluate performance.

### Import required libraries and Define the custom environment - 2 Marks

In [ ]:
# Import statements
import numpy as np
import gym
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv

class TreasureHuntFrozenLakeEnv(FrozenLakeEnv):
    def __init__(self, desc=None, map_name="5x5", is_slippery=True):
        # Define the 5x5 map with treasures (T)
        desc = np.array([
            "SFHHT",
            "FHFFF",
            "FFFTF",
            "TFFHF",
            "FFFFG"
        ], dtype="c")
        super().__init__(desc=desc, map_name=map_name, is_slippery=is_slippery)
        self.treasure_positions = {(0, 4), (2, 3), (3, 0)}  # Treasure positions

    def step(self, action):
        obs, reward, done, info = super().step(action)
        row, col = divmod(obs, self.ncol)

        # If the agent steps on a treasure
        if (row, col) in self.treasure_positions:
            reward += 5  # Award +5 for treasure
            self.desc[row, col] = b'F'  # Convert treasure tile to frozen tile
            self.treasure_positions.remove((row, col))
        return obs, reward, done, info


In [ ]:
# Custom environment to create the given grid and respective functions that are required for the problem

#Include functions to take an action, get reward, to check if episode is over

### Value Iteration Algorithm - 1 Mark

In [ ]:
def value_iteration(env, gamma=0.99, theta=1e-6):
    value_function = np.zeros(env.nS)  # Initialize value function

    while True:
        delta = 0
        for state in range(env.nS):
            if state in env.holes or state == env.goal:
                continue  # Skip terminal states
            v = value_function[state]
            value_function[state] = max(compute_q_value(env, value_function, state, action, gamma)
                                        for action in range(env.nA))
            delta = max(delta, abs(v - value_function[state]))

        if delta < theta:  # Stop if value function converges
            break
    return value_function


### Policy Improvement Function - 1 Mark

In [ ]:
def policy_improvement(env, value_function, gamma=0.99):
    policy = np.zeros(env.nS, dtype=int)

    for state in range(env.nS):
        if state in env.holes or state == env.goal:
            continue
        policy[state] = np.argmax([compute_q_value(env, value_function, state, action, gamma)
                                   for action in range(env.nA)])
    return policy

def compute_q_value(env, value_function, state, action, gamma):
    q_value = 0
    for prob, next_state, reward, done in env.P[state][action]:
        q_value += prob * (reward + gamma * value_function[next_state] * (not done))
    return q_value


### Print the Optimal Value Function

In [ ]:
def print_value_function(value_function, env):
    print("Optimal Value Function:")
    grid = np.round(value_function.reshape(env.nrow, env.ncol), 2)
    print(grid)


### Visualization of the learned optimal policy - 1 Mark

In [ ]:
def visualize_policy(policy, env):
    actions = ['←', '↓', '→', '↑']  # Left, Down, Right, Up
    policy_grid = np.array([actions[action] for action in policy]).reshape(env.nrow, env.ncol)
    print("Learned Optimal Policy:")
    print(policy_grid)


### Evaluate the policy - 1 Mark

In [ ]:
def evaluate_policy(env, policy, episodes=100):
    total_reward = 0
    for _ in range(episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action = policy[state]
            state, reward, done, _ = env.step(action)
            episode_reward += reward
        total_reward += episode_reward
    avg_reward = total_reward / episodes
    print(f"Average Reward over {episodes} episodes: {avg_reward}")


### Main Execution

In [ ]:
if __name__ == "__main__":
    # Initialize the custom environment
    env = TreasureHuntFrozenLakeEnv()

    # Step 1: Perform Value Iteration
    optimal_value_function = value_iteration(env)

    # Step 2: Improve Policy based on the Value Function
    optimal_policy = policy_improvement(env, optimal_value_function)

    # Step 3: Print the Optimal Value Function
    print_value_function(optimal_value_function, env)

    # Step 4: Visualize the Learned Policy
    visualize_policy(optimal_policy, env)

    # Step 5: Evaluate the Policy
    evaluate_policy(env, optimal_policy)
